In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

l=["https://zeenews.india.com/latest-news","https://zeenews.india.com/india","https://zeenews.india.com/lok-sabha-elections-2024","https://zeenews.india.com/cricket","https://zeenews.india.com/entertainment","https://zeenews.india.com/lifestyle","https://zeenews.india.com/business","https://zeenews.india.com/world","https://zeenews.india.com/auto","https://zeenews.india.com/viral"]
t=[]
des=[]
g=[]

# Initialize the Selenium WebDriver (ensure you have the appropriate driver installed)
driver = webdriver.Firefox()  # Or any other WebDriver you prefer


scrolls = 1  # Change the number of scrolls as needed
for i in l:
    for n in range(scrolls):
        driver.get(i)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        div_tags_titles = soup.find_all('div', class_='news_description desc-title morenews-title')
        div_tags_desc = soup.find_all('div', class_='more-news-desc')
        div_date = soup.find_all('div',class_='updated-timestamp')
        
        for div_tag in div_tags_titles:
            anchor_tags = div_tag.find_all('a')
            titles = [tag.text.strip() for tag in anchor_tags]

            for title in titles:
                t.append(title)
        for div in div_tags_desc:
            desc_tags = div.find_all('a')
            desc = [tag.text.strip() for tag in desc_tags]
            for d in desc:
                des.append(d)
                g.append(i)
                g_cleaned = [url.replace('https://zeenews.india.com/', '') for url in g]  
                
    
df = pd.DataFrame({'Title': t, 'Description': des,'Genres': g_cleaned})
df['Description'] = df['Description'].fillna('No Description Available').replace('', 'No Description Available')
print(df)
df.to_csv("News2.csv")
        
WebDriverWait(driver, 10)

# Close the WebDriver
driver.quit()


C:\Users\AGL\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\AGL\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\AGL\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


                                                 Title  \
0    High Blood Pressure In Childhood Linked To 4x ...   
1    Meet Roz Kelly Morkel, LSG Coach Morne Morkel'...   
2    Lok Sabha Elections 2024: Banks Will Be Closed...   
3    IPL 2024: Why Matheesha Pathirana Did Not Play...   
4    Rajgarh LS Seat: Ex-CM Digvijaya Singh Fights ...   
..                                                 ...   
175  Rs 4,000 Bill For 2 Rolls And Coffees Baffles ...   
176  Bizarre: Thief Sells iPhone For 1 Plate Pav Bh...   
177  Polish Man Tours India With His Mom, Shares He...   
178  'Installment Period Bhi To Batao': SIP Poster ...   
179  Biker's Witty Reason For Booking Rapido Goes V...   

                                           Description       Genres  
0    Hypertension is known to affect one in every 1...  latest-news  
1    Step into the enchanting world of cricket wher...  latest-news  
2    Several banks across India will remain closed ...  latest-news  
3    IPL 2024: Why Math

In [2]:
X = np.array(df.Description)

In [3]:
text_data = X
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(text_data, show_progress_bar=True)

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
embed_data = embeddings

In [5]:
X = np.array(embed_data)

In [6]:
cos_sim_data = pd.DataFrame(cosine_similarity(X))

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

def give_recommendations(search_term, print_recommendation=False, print_recommendation_plots=False, print_genres=False):
    # Filter news articles containing the search term
    relevant_indices = df[df['Description'].str.contains(search_term, case=False, na=False)].index
    
    if len(relevant_indices) == 0:
        return "No relevant news found."

    # Calculate TF-IDF vectors
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['Description'])

    # Calculate cosine similarity between search term and all news articles
    search_vector = tfidf_vectorizer.transform([search_term])
    cosine_similarities = cosine_similarity(search_vector, tfidf_matrix).flatten()

    # Get indices of top similar news articles
    index_recomm = cosine_similarities.argsort()[-6:-1][::-1]

    news_recomm = df['Title'].iloc[index_recomm].values
    result = {'News': news_recomm, 'Index': index_recomm}

    if print_recommendation:
        print(f"Latest News related to '{search_term}':")
        for idx, news in enumerate(news_recomm, start=1):
            print(f"{idx}. {news}")

    if print_recommendation_plots:
        print(f"\nShort Description of above latest news related to '{search_term}':")
        for idx, news_idx in enumerate(index_recomm, start=1):
            plot_q = df['Description'].iloc[news_idx]
            print(f"{idx}. {plot_q}")

    if print_genres:
        print(f"\nGenres of above latest news related to '{search_term}':")
        for idx, news_idx in enumerate(index_recomm, start=1):
            genres_q = df['Genres'].iloc[news_idx]
            print(f"{idx}. {genres_q}")

# News Bot

In [8]:
s = str(input("Enter the specific word for which you have to find latest news if it is available : "))
rec = True
d = str(input("Description for above news is needed??\nType yes or no :"))
if d=="yes":
    d=True
else:
    d=False
    
gen = str(input("Genres for above news is needed??\nType yes or no : "))
if gen=="yes":
    gen=True
else:
    gen=False
    

give_recommendations(s,rec,d,gen)

Enter the specific word for which you have to find latest news if it is available : elections
Description for above news is needed??
Type yes or no :yes
Genres for above news is needed??
Type yes or no : yes
Latest News related to 'elections':
1. Lok Sabha Elections 2024: Banks Will Be Closed In These States On May 7
2. K'taka Sex Abuse Case:BJP Chief Denies Getting Any Letter Regarding Allegation
3. Can Mumbai Indians Qualify For Playoffs After Defeat Against KKR In IPL 2024?
4. Pushpa Pushpa: The First Single From Allu Arjun's 'Pushpa 2' To Drop Today!
5. 'Pushpa Pushpa' Song Out: First Song From Allu Arjun's 'Pushpa 2' Out Now

Short Description of above latest news related to 'elections':
1. Several banks across India will remain closed due to Phase 3 of the Lok Sabha Elections 2024 on May 7.
2. Karnataka BJP President BY Vijyendra on Saturday denied the knowledge of sexual abuse allegations against Hassan JD(S) MP Prajwal Revanna and emphasised that issues can not adversely impact